In [5]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
train = pd.read_csv('strain0.csv')
test = pd.read_csv('stest0.csv')
result=pd.read_csv('sample_submission_2.csv')
del train['Unnamed: 0']
del test['Unnamed: 0']
print (train.shape)
print (test.shape)

(40000, 213)
(120000, 149)


In [8]:
#definite WMAE
def weighted_mae(y_true, y_pred, weights):
    """
    Weighted Mean Absolute Error
    """
    return np.mean(weights * np.abs(y_true - y_pred))
    
import sklearn
from sklearn.metrics import make_scorer
WMAE1 = sklearn.metrics.make_scorer(weighted_mae, weights = train.iloc[:,211], greater_is_better=False)
WMAE2 = sklearn.metrics.make_scorer(weighted_mae, weights = train.iloc[:,212], greater_is_better=False)

In [9]:
#find Lasso alphas
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
model=Lasso()
lalpha = np.zeros(62)

    
for i in range(0,62):
    if i==60:
        grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas),scoring=WMAE2,cv=5)
        grid.fit(train.iloc[:,0:149],train.iloc[:,149+i])
        print('Best score (LassoRegressor): {}'.format(grid.best_score_))
        print('Best parameters: {}'.format(grid.best_estimator_.alpha))
        lalpha[i]=grid.best_estimator_.alpha
    elif i==61:
        grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas),scoring=WMAE2,cv=5)
        grid.fit(train.iloc[:,0:149],train.iloc[:,149+i])
        print('Best score (LassoRegressor): {}'.format(grid.best_score_))
        print('Best parameters: {}'.format(grid.best_estimator_.alpha))
        lalpha[i]=grid.best_estimator_.alpha
    else:
        grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas),scoring=WMAE1,cv=5)
        grid.fit(train.iloc[:,0:149],train.iloc[:,149+i])
        print('Best score (LassoRegressor): {}'.format(grid.best_score_))
        print('Best parameters: {}'.format(grid.best_estimator_.alpha))
        lalpha[i]=grid.best_estimator_.alpha

Best score (LassoRegressor): -867.655120527
Best parameters: 1.0
Best score (LassoRegressor): -934.271292003
Best parameters: 0.0001
Best score (LassoRegressor): -903.889251388
Best parameters: 1.0
Best score (LassoRegressor): -868.60996525
Best parameters: 1.0
Best score (LassoRegressor): -866.717469941
Best parameters: 0.0001
Best score (LassoRegressor): -861.27282121
Best parameters: 1.0
Best score (LassoRegressor): -910.515895341
Best parameters: 1.0
Best score (LassoRegressor): -889.365727265
Best parameters: 1.0
Best score (LassoRegressor): -857.014331604
Best parameters: 1.0
Best score (LassoRegressor): -868.007691315
Best parameters: 1.0
Best score (LassoRegressor): -861.460557682
Best parameters: 1.0
Best score (LassoRegressor): -895.91287351
Best parameters: 1.0
Best score (LassoRegressor): -874.408869984
Best parameters: 1.0
Best score (LassoRegressor): -856.825895454
Best parameters: 0.0001
Best score (LassoRegressor): -868.090348576
Best parameters: 1.0
Best score (LassoRe

KeyboardInterrupt: 

In [16]:
print lalpha

[  1.00000000e+00   1.00000000e-04   1.00000000e+00   1.00000000e+00
   1.00000000e-04   1.00000000e+00   1.00000000e+00   1.00000000e+00
   1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
   1.00000000e+00   1.00000000e-04   1.00000000e+00   1.00000000e-04
   1.00000000e-04   1.00000000e+00   1.00000000e-04   1.00000000e+00
   1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
   1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e-04
   1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
   1.00000000e-04   1.00000000e+00   1.00000000e+00   1.00000000e+00
   1.00000000e-04   1.00000000e+00   1.00000000e+00   1.00000000e+00
   1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
   1.00000000e+00   1.00000000e+00   1.00000000e-04   1.00000000e+00
   1.00000000e-04   1.00000000e+00   1.00000000e+00   1.00000000e-04
   1.00000000e+00   1.00000000e+00   1.00000000e+00   1.00000000e+00
   1.00000000e+00   1.00000000e+00

In [18]:
#Lasso model
#1773.60505，优于ridge
#score:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
def Lassore(xtrain,ytrain,X_test,alpha):
    lasso = Lasso(alpha=alpha)
    lasso.fit(xtrain, ytrain)
    p=lasso.predict(X_test)
    return p

    
laspredict = np.zeros((120000, 62))
for i in range(0,62):
    if i==60:
        laspredict[:,i]=Lassore(train.iloc[:,0:149],train.iloc[:,149+i],test,lalpha[i])
    elif i==61:
        laspredict[:,i]=Lassore(train.iloc[:,0:149],train.iloc[:,149+i],test,lalpha[i])
    else:
        laspredict[:,i]=Lassore(train.iloc[:,0:149],train.iloc[:,149+i],test,lalpha[i])
        
        
resultlas=np.zeros(7440000)
for j in range(0,120000):
    resultlas[62*j:62+62*j]=laspredict[j,:]

result.iloc[0:,1]=resultlas
result.to_csv('result-lasso-0.csv',index=False)

In [ ]:
#ridge model
#score:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
def Ridgere(xtrain,ytrain,X_test,score):
    alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
    model = Ridge()
    grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas),scoring=score,cv=5)
    grid.fit(xtrain,ytrain)
    print('Best score (RidgeRegressor): {}'.format(grid.best_score_))
    print('Best parameters: {}'.format(grid.best_estimator_.alpha))
    ridge = Ridge(alpha=grid.best_estimator_.alpha)
    ridge.fit(xtrain, ytrain)
    p=ridge.predict(X_test)
    return p

    
ridpredict = np.zeros((120000, 62))
for i in range(0,62):
    if i==60:
        ridpredict[:,i]=Ridgere(train.iloc[:,0:149],train.iloc[:,149+i],test,WMAE2)
    elif i==61:
        ridpredict[:,i]=Ridgere(train.iloc[:,0:149],train.iloc[:,149+i],test,WMAE2)
    else:
        ridpredict[:,i]=Ridgere(train.iloc[:,0:149],train.iloc[:,149+i],test,WMAE1)
        
        
resultrid=np.zeros(7440000)
for j in range(0,120000):
    resultrid[62*j:62+62*j]=ridpredict[j,:]
result.iloc[0:,1]=resultrid
result.to_csv('result-ridge-0.csv',index=False)

In [ ]:
#ExtraTreesRegressor
#score
from sklearn.ensemble import ExtraTreesRegressor

extpredictm = np.zeros((120000, 62))
for i in range(0,62):
    if i==60:
        weight=train.iloc[:,212].as_matrix()
        clf = ExtraTreesRegressor(n_estimators=1000, max_depth=35,
                                    max_features=149, min_samples_leaf=20)
        clf.fit(train.iloc[:,0:149],train.iloc[:,149+i],weight)
        extpredictm[:,i]=clf.predict(test)
    elif i==61:
        weight=train.iloc[:,212].as_matrix()
        clf = ExtraTreesRegressor(n_estimators=1000, max_depth=35,
                                    max_features=149, min_samples_leaf=20)
        clf.fit(train.iloc[:,0:149],train.iloc[:,149+i],weight)
        extpredictm[:,i]=clf.predict(test)
    else:
        weight=train.iloc[:,211].as_matrix()
        clf = ExtraTreesRegressor(n_estimators=950, max_depth=20,
                                    max_features=149, min_samples_leaf=15)
        clf.fit(train.iloc[:,0:149],train.iloc[:,149+i],weight)
        extpredictm[:,i]=clf.predict(test)

resultext=np.zeros(7440000)
for j in range(0,120000):
    resultext[62*j:62+62*j]=extpredictm[j,:]
result.iloc[0:,1]=resultext
result.to_csv('result-extra2-0.csv',index=False)